# perform authorization
[apt-key](https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force)

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
path="drive/capstone/"

In [0]:
import pandas as pd
import numpy as np

import re
import os
import csv

import matplotlib.pyplot as plt

# CCL

In [0]:
CCL = pd.read_csv(path+"datas/pred_CCL.csv")

In [0]:
CCL["Date"] = pd.to_datetime(CCL.Date, format="%Y-%m-%d", errors='ignore')

In [0]:
CCL["CCL_lastweek"] =  CCL.CCL.shift(-1)

In [0]:
CCL["change_CCL"] = (CCL.CCL/CCL.CCL_lastweek-1)*100

In [0]:
CCL["pred_change_CCL"] = (CCL.pred_CCL/CCL.CCL_lastweek-1)*100

In [0]:
CCL.set_index("Date", inplace=True)

In [11]:
CCL.head()

,CCL,pred_CCL,CCL_lastweek,change_CCL,pred_change_CCL
Date,,,,,
2017-04-30,155.45,156.692913,154.57,0.569321,1.373432
2017-04-23,154.57,155.880065,152.77,1.178242,2.035783
2017-04-16,152.77,155.433498,152.57,0.131087,1.876842
2017-04-09,152.57,155.122248,152.18,0.256275,1.933400
2017-04-02,152.18,154.011629,150.89,0.854927,2.068811


# STOCK

In [12]:
for i in os.listdir(path+"stock_price"):
  code = i[:4]
  stock = pd.read_csv(path+"stock_price/{}.HK.csv".format(code))
  stock["Date"] = pd.to_datetime(stock.Date, format="%Y-%m-%d", errors='ignore')
  stock.set_index("Date", inplace=True)
  stock = (stock.resample('W', 'last')["Close"]/stock.resample('W', 'first')["Open"] - 1)*100
  stock = stock.to_frame(name=code)
  CCL = pd.merge(CCL,stock, left_index=True, right_index=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).first()
  


In [0]:
CCL.dropna(inplace=True)

In [14]:
CCL.head()

,CCL,pred_CCL,CCL_lastweek,change_CCL,pred_change_CCL,0019,0003,0945,0390,0066,...,1200,0432,0083,0054,0041,0020,0001,0017,0012,0016
Date,,,,,,,,,,,,,,,,,,,,,
2017-04-30,155.45,156.692913,154.57,0.569321,1.373432,-0.728953,0.517268,1.412635,-2.511078,0.674155,...,-0.483092,1.302932,1.540832,2.413797,1.846955,-2.177416,2.913136,0.937500,0.612409,0.777197
2017-04-23,154.57,155.880065,152.77,1.178242,2.035783,-1.986755,-2.162915,-0.962965,-3.458213,-0.894859,...,-2.336449,0.000000,-3.880597,-0.173010,-2.699239,-2.843600,-2.460731,-3.115578,-1.507718,-2.510460
2017-04-16,152.77,155.433498,152.57,0.131087,1.876842,-2.880922,-0.378472,-0.296292,-3.098592,1.940646,...,-2.252252,0.323625,-0.737463,-1.530609,3.050390,0.157726,-1.659750,2.579979,2.142985,1.445583
2017-04-09,152.57,155.122248,152.18,0.256275,1.933400,-0.192186,1.144784,-1.454545,2.446043,0.000000,...,6.280193,-0.321543,-1.457726,0.512821,3.708786,3.074437,-0.104664,0.937500,1.659584,1.739130
2017-04-02,152.18,154.011629,150.89,0.854927,2.068811,0.000000,1.568757,1.776457,-6.836461,1.866987,...,-2.816901,-0.638978,-1.589595,0.000000,-0.272859,0.000000,-0.726900,-2.049180,-0.103834,0.087642


In [15]:
CCL.shape

(456, 33)

In [16]:
CCL.dtypes

CCL                float64
pred_CCL           float64
CCL_lastweek       float64
change_CCL         float64
pred_change_CCL    float64
0019               float64
0003               float64
0945               float64
0390               float64
0066               float64
0062               float64
0733               float64
0488               float64
0050               float64
0127               float64
0014               float64
0688               float64
0158               float64
3323               float64
0914               float64
0002               float64
0034               float64
0683               float64
1200               float64
0432               float64
0083               float64
0054               float64
0041               float64
0020               float64
0001               float64
0017               float64
0012               float64
0016               float64
dtype: object

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

In [18]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

features = "\")+Q(\"".join([i[0:4] for i in os.listdir(path+"stock_price")])

result = smf.ols("change_CCL~Q(\"{}\")".format(features) ,data = CCL).fit()

/usr/local/lib/python3.6/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [19]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:             change_CCL   R-squared:                       0.168
Model:                            OLS   Adj. R-squared:                  0.113
Method:                 Least Squares   F-statistic:                     3.078
Date:                Tue, 22 May 2018   Prob (F-statistic):           5.00e-07
Time:                        08:27:49   Log-Likelihood:                -539.94
No. Observations:                 456   AIC:                             1138.
Df Residuals:                     427   BIC:                             1257.
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1743      0.039      4.421      0.0

In [0]:
select_columns = ["0012","0017","0683","3323","0158","0062","0066"]
features = "\")+Q(\"".join(select_columns)

result = smf.ols("change_CCL~Q(\"{}\")".format(features) ,data = CCL).fit()

In [21]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:             change_CCL   R-squared:                       0.120
Model:                            OLS   Adj. R-squared:                  0.106
Method:                 Least Squares   F-statistic:                     8.692
Date:                Tue, 22 May 2018   Prob (F-statistic):           5.05e-10
Time:                        08:27:51   Log-Likelihood:                -552.82
No. Observations:                 456   AIC:                             1122.
Df Residuals:                     448   BIC:                             1155.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1789      0.039      4.608      0.0

In [0]:
select_columns.append('change_CCL')
select_columns.append("pred_change_CCL")
data = CCL[select_columns].sort_index(ascending=True)

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [24]:
data.head()

,0012,0017,0683,3323,0158,0062,0066,change_CCL,pred_change_CCL
Date,,,,,,,,,
2008-06-15,-3.794942,-5.405594,-13.700308,-5.000000,-1.249998,-1.666661,-1.886792,0.013656,-3.525899
2008-06-22,1.964569,-4.148195,-1.814297,10.065359,-1.538462,-3.380285,-2.294459,-0.846532,-5.252630
2008-06-29,-4.373266,-2.944327,-9.130437,-8.982036,1.984737,-1.884064,-2.745102,1.046544,-6.784890
2008-07-06,-6.969771,-11.287783,-4.470593,-5.194805,1.347296,-4.985755,-4.339250,-0.831289,-7.983924
2008-07-13,1.307011,5.249146,-1.711494,12.328767,-0.437961,-4.579022,2.070393,-0.535935,-7.502853


In [25]:
data.iloc[0:4,7].values

array([ 0.01365561, -0.84653195,  1.04654365, -0.83128918])

In [0]:
weight_o = np.array([0.002073, 0.006396,0.047383,0.021803,0.359315,0.363075, 0.199954])

In [28]:
if data.iloc[4,8]>0:
  flag = 1
else:
  flag=-1

False

In [0]:
y_pred = []
y_valid = []
weights = []
for i in range(data.shape[0]-4):
  X_train = data.iloc[i:i+4,0:7].values
  X_test = data.iloc[i+4,0:7].values.reshape(1,7)
  y_train = data.iloc[i:i+4,7].values
  y_test = data.iloc[i+4,7]
  model = LinearRegression()
  model.fit(X_train,y_train)
  
  #TODO beta and rebalancing order backtest
  beta = model.coef_
  e_beta = np.exp(beta)/np.sum(np.exp(beta))
  chan = e_beta*data.iloc[i+4,8]/100
  weight = weight_o*(1+chan)/np.sum(weight_o*(1+chan))
  
  #if data.iloc[4,8]>0:
  #  flag = 1
  #else:
  #  flag = -1
  #e_beta = np.exp(flag*beta)/np.sum(np.exp(flag*beta))
  #weight = e_beta
  
  pred = model.predict(X_test)
  y_pred.append(pred)
  y_valid.append(y_test)
  weights.append([i for i in weight])

In [30]:
len(weights)

452

In [31]:
data.index[4:]

DatetimeIndex(['2008-07-13', '2008-07-20', '2008-07-27', '2008-08-03',
               '2008-08-10', '2008-08-17', '2008-08-24', '2008-08-31',
               '2008-09-07', '2008-09-14',
               ...
               '2017-02-26', '2017-03-05', '2017-03-12', '2017-03-19',
               '2017-03-26', '2017-04-02', '2017-04-09', '2017-04-16',
               '2017-04-23', '2017-04-30'],
              dtype='datetime64[ns]', name='Date', length=452, freq=None)

In [32]:
data.columns[:-1]

Index(['0012', '0017', '0683', '3323', '0158', '0062', '0066', 'change_CCL'], dtype='object')

In [0]:
df_weights = pd.DataFrame.from_records([tuple(l) for l in weights], columns=data.columns[:-2], index=data.index[4:])

In [34]:
df_weights.head()

,0012,0017,0683,3323,0158,0062,0066
Date,,,,,,,
2008-07-13,0.141978,0.125348,0.144159,0.162111,0.138761,0.141662,0.145981
2008-07-20,0.131757,0.132655,0.158358,0.155334,0.144465,0.131504,0.145928
2008-07-27,0.142855,0.123270,0.144545,0.163230,0.136088,0.141373,0.148638
2008-08-03,0.142359,0.141920,0.142680,0.143684,0.143703,0.143040,0.142614
2008-08-10,0.142386,0.138416,0.146416,0.145876,0.143722,0.142285,0.140899


In [35]:
df_weights.index

DatetimeIndex(['2008-07-13', '2008-07-20', '2008-07-27', '2008-08-03',
               '2008-08-10', '2008-08-17', '2008-08-24', '2008-08-31',
               '2008-09-07', '2008-09-14',
               ...
               '2017-02-26', '2017-03-05', '2017-03-12', '2017-03-19',
               '2017-03-26', '2017-04-02', '2017-04-09', '2017-04-16',
               '2017-04-23', '2017-04-30'],
              dtype='datetime64[ns]', name='Date', length=452, freq=None)

In [0]:
df_weights.to_csv(path+"weights.csv")